In [1]:
"""
MSE baixo significa que o modelo está produzindo previsões próximas aos valores reais,
enquanto um alto R2 indica que a maior parte da variação nos dados está sendo explicada 
pelo modelo.

- Modelo está sendo capaz de prever resultados com alta precisão.

Valores reais.
MSE LSTM: 0.15506279163361772
R2 LSTM: 0.8398106636127365 

"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score


df = pd.read_csv('https://raw.githubusercontent.com/cardosource/OraculoAI/main/2022.csv', header=None)
data = np.array(df).reshape(-1, 1)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

 
lookback = 6  # número de passos de tempo para trás
X = []
y = []
for i in range(lookback, len(data)):
    X.append(data_scaled[i-lookback:i, 0])
    y.append(data_scaled[i, 0])
X, y = np.array(X), np.array(y)

train_size = int(len(X) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# construção do modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(lookback, 1)))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=200, verbose=0)

forecast_lstm = []
current_lstm = X_test[-1]
for i in range(36):
    x_input = np.reshape(current_lstm, (1, lookback, 1))
    yhat = model_lstm.predict(x_input, verbose=0)
    forecast_lstm.append(yhat[0])
    current_lstm = np.append(current_lstm[1:], yhat[0])
forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1, 1))

# Avaliação do modelo LSTM
print('MSE LSTM:', mean_squared_error(y_test, model_lstm.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))))
print('R2 LSTM:', r2_score(y_test, model_lstm.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))))
 
print('Previsão LSTM:', forecast_lstm.flatten())


2023-05-11 19:49:59.718463: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-11 19:50:00.122190: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-11 19:50:12.498968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


7/7 [==============================] - 0s 38ms/step
MSE LSTM: 0.15026947110553945
7/7 [==============================] - 0s 24ms/step
R2 LSTM: 0.8447624565373638
Previsão LSTM: [55.87656   18.870453  26.946827  33.113792  41.399174  49.255474
 56.982697   8.177263  18.74014   26.575218  36.462837  44.46648
 53.30632    3.6317232 15.040325  24.519415  35.296898  43.315376
 51.909676   6.8613944 16.929033  25.294193  35.480072  43.62959
 52.067444   4.8886657 15.596995  24.621546  35.25117   43.35172
 51.837036   6.195688  16.452911  25.025257  35.3654    43.509483 ]
